In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import torch

In [ ]:
from thesis.data_management.data_classes import UnbatchedFlameParams
from jaxtyping import Float


def flame_params_to_code(params: UnbatchedFlameParams) -> Float[torch.Tensor, 'batch time 103']:
    expr = params.expr
    jaw = params.jaw
    return torch.cat([expr, jaw], dim=-1).unsqueeze(0)


a = sm.flame_params[:]
code = flame_params_to_code(a)

In [ ]:
code.shape

In [ ]:
from thesis.data_management import SequenceManager

sm = SequenceManager(3)
sm.cameras[1]

In [ ]:
from thesis.video_utils import side_by_side

seq = 8
side_by_side(
    # f'tmp/pred/2dgs_full_res_500k_overnight_rigging_large_lpips/flame/sequence_{seq}.mp4',
    f'tmp/gt/masked/sequence_{seq}.mp4',
    f'tmp/pred/2dgs_full_res_500k_overnight_rigging_large_lpips/audio/sequence_{seq}.mp4',
    f'tmp/pred/2dgs_full_res_500k_overnight_rigging_large_lpips/side_by_side/sequence_{seq}.mp4',
)

In [ ]:
from thesis.dataset_statistics import plot_dataset_stats

fig = plot_dataset_stats()
fig.write_image("figures/dataset_statistics.pdf")
fig.show()


In [ ]:
a = 'a85'
int(a)

In [ ]:
from thesis.evaluation import evaluate

gt_dir = 'tmp/gt/masked'
pred_dir = 'tmp/pred/2dgs_full_res_500k_overnight_rigging_large_lpips/audio'

evaluate(gt_dir, pred_dir, device='cuda', sequences=[i for i in range(80, 82)])

In [ ]:
from thesis.evaluation import evaluate

gt_dir = 'tmp/gt/masked/other_guy'
pred_dir = 'tmp/pred/other_guy/flame'

evaluate(gt_dir, pred_dir, device='cuda', sequences=[9, 10])

In [ ]:
from thesis.evaluation import plot_all_visualizations

for key, value in plot_all_visualizations(pred_dir).items():
    print(key)
    value.show()


In [ ]:
from thesis.data_management import SequenceManager
from thesis.constants import TEST_CAMS, OTHER_GUY_DATA_DIR

# i = np.random.randint(1, 10)
sm = SequenceManager(5, data_dir=OTHER_GUY_DATA_DIR, cameras=TEST_CAMS)
# j = np.random.randint(0, len(sm))
img = sm.images[66][0].numpy()

import matplotlib.pyplot as plt

plt.imshow(img)
plt.show()

In [ ]:
sm.flame_params[66:67]

In [ ]:
from thesis.constants import TEST_CAMS
from thesis.data_management import SequenceManager

j = np.random.randint(80, 101)
sm = SequenceManager(j, cameras=TEST_CAMS)
i = np.random.randint(0, len(sm))
img = sm.images[i][0]
cut_pixels = 350
img = img[:-cut_pixels].numpy()

import matplotlib.pyplot as plt

plt.imshow(img)
plt.show()

In [ ]:
from thesis.video_utils import load_video, get_audio_path
from thesis.data_management import SequenceManager

# window = 21
# 80,81,85
# 80: 114, 94, 93
# 81: 126, 106, 105
# 85: 136, 117, 115 (???)
# 92: 96, 76, 75
# 93: 124, 105, 103 (???)
# 95: 126, 106, 105
# 97: 117, 97, 96
# 100: 150, 130, 129

sequence = 85
audio_path = get_audio_path(sequence)
gt_path = f'tmp/gt/masked/sequence_{sequence}.mp4'
gt_video = load_video(gt_path)
gt_video = torch.from_numpy(gt_video)

pred_path = f'tmp/pred/2dgs_full_res_500k_overnight_rigging_large_lpips/flame/sequence_{sequence}.mp4'
pred_video = load_video(pred_path)
pred_video = torch.from_numpy(pred_video)

print(gt_video.shape, pred_video.shape)

sm = SequenceManager(sequence)
sm.audio_features[:].shape

In [ ]:
n_frames = 100
window_size = 21
l = range(window_size // 2, n_frames - window_size//2)
len(list(l))

In [ ]:
from thesis.video_utils import get_audio_path
import soundfile as sf

audio_path = get_audio_path(sequence)
audio, sr = sf.read(audio_path)

In [ ]:
sm.audio_features[:].shape[0] / (audio.shape[0] / sr)

In [ ]:
from thesis.evaluation import evaluate

gt_dir = 'tmp/gt/masked'
pred_dir = 'tmp/pred/2dgs_full_res_500k_overnight_rigging_large_lpips/flame'

evaluate(gt_dir, pred_dir, device='cuda', sequences=[i for i in range(80, 102)])

In [ ]:
from thesis.evaluation import plot_all_visualizations

for key, value in plot_all_visualizations(pred_dir).items():
    print(key)
    value.show()


In [ ]:
from thesis.evaluation import _EvaluationComputer

ec = _EvaluationComputer()
ec.cuda()
ec.forward(gt_video, pred_video, device='cuda')

In [ ]:
from thesis.evaluation import fov_video_vdp

score, heatmap = fov_video_vdp(gt_video=gt_video, pred_video=pred_video)

In [ ]:
from thesis.video_utils import save_video

save_video(heatmap, 'tmp/heatmap.mp4', audio_path=audio_path)

In [ ]:
from thesis.video_utils import side_by_side
import os

sequence = 100
name = '2dgs_full_res_500k_noisy_audio/flame'
gt_path = f'tmp/gt/sequence_{sequence}.mp4'
pred_path = f'tmp/pred/{name}/sequence_{sequence}.mp4'
output_dir = f'tmp/side_by_side/{name}'
os.makedirs(output_dir, exist_ok=True)
output_path = f'{output_dir}/sequence_{sequence}.mp4'
side_by_side(gt_path, pred_path, output_path)

In [ ]:
from thesis.flame import FlameHeadWithInnerMouth
from thesis.data_management import SequenceManager, UnbatchedFlameParams

t = 218

sm = SequenceManager(3)
flame_head = FlameHeadWithInnerMouth()
flame_head = flame_head.cuda()
# vertices = torch.load('tmp.pt', weights_only=True)
flame_params = sm.flame_params[t:t + 1]
flame_params = UnbatchedFlameParams(
    shape=flame_params.shape.cuda(),
    expr=flame_params.expr.cuda(),
    neck=flame_params.neck.cuda(),
    #neck=torch.ones_like(flame_params.neck).cuda() * 0.5,
    jaw=flame_params.jaw.cuda(),
    eye=flame_params.eye.cuda(),
    scale=flame_params.scale.cuda(),
)
flame_vertices = flame_head(flame_params)[0]

In [ ]:
flame_vertices.shape

In [ ]:
from thesis.gaussian_splatting.rigged_gaussian_splatting import RiggedGaussianSplatting

checkpoint_path = 'tb_logs/rigid_gs/static/static_initialization/version_3/checkpoints/epoch=29-step=135000.ckpt'

model = RiggedGaussianSplatting.load_from_checkpoint(checkpoint_path, ckpt_path=checkpoint_path)

In [ ]:
checkpoint = torch.load(checkpoint_path)

In [ ]:
from thesis.data_management import SingleSequenceDataset
from thesis.constants import TRAIN_CAMS

time_step = 218
sequence = 3

train_set = SingleSequenceDataset(
    cameras=TRAIN_CAMS,
    sequence=sequence,
    start_idx=time_step,
    end_idx=time_step + 1,
    n_cameras_per_frame=1,
    length_multiplier=500,
)

In [ ]:
img = train_set[0].image

from matplotlib import pyplot as plt

plt.imshow(img[0])
plt.show()

In [ ]:
from thesis.deformation_field.rigging_params import RiggingParams

model = RiggingParams()

In [ ]:
blue_idx = flame_head.mask.v.teeth
green_idx = flame_head.inner_mouth_indices

import plotly.graph_objects as go

fig = go.Figure()
permutation = [0, 2, 1]
vertices = flame_vertices.detach().cpu().numpy()[:, permutation]  # has shape (n_vertices, 3)
faces = flame_head.faces.detach().cpu().numpy()[:, permutation]  # has shape (n_faces, 3)

blue_idx = blue_idx.cpu().numpy()
blue_points = vertices[blue_idx]
green_idx = green_idx.cpu().numpy()
green_points = vertices[green_idx]

fig.add_trace(
    go.Mesh3d(
        x=vertices[:, 0],
        y=vertices[:, 1],
        z=vertices[:, 2],
        i=faces[:, 0],
        j=faces[:, 1],
        k=faces[:, 2],
        color='lightpink',
        opacity=0.50,
    ))

# # plot points
# fig.add_trace(
#     go.Scatter3d(
#         x=vertices[:, 0],
#         y=vertices[:, 1],
#         z=vertices[:, 2],
#         mode='markers',
#         marker=dict(
#             size=2,
#             color='red',
#         ),
#     ))

# plot blue points
fig.add_trace(
    go.Scatter3d(
        x=blue_points[:, 0],
        y=blue_points[:, 1],
        z=blue_points[:, 2],
        mode='markers',
        marker=dict(color='blue',),
    ))

# plot green points
fig.add_trace(
    go.Scatter3d(
        x=green_points[:, 0],
        y=green_points[:, 1],
        z=green_points[:, 2],
        mode='markers',
        marker=dict(color='green',),
    ))

In [ ]:
# plot flame vertices
lower = True
upper = True
show_lips = False
connect_rings = False
inside_mouth_ring_plot = True

vertices = flame_vertices.cpu().numpy()
lips = flame_vertices[flame_head.mask.v.lips].cpu().numpy()  # red

lips_outside_ring_lower = flame_vertices[  # green
    flame_head.mask.v.lip_outside_ring_lower].cpu().detach().numpy()
lips_outside_ring_upper = flame_vertices[
    flame_head.mask.v.lip_outside_ring_upper].cpu().detach().numpy()
outside_ring = []
if lower:
    outside_ring.append(lips_outside_ring_lower)
if upper:
    outside_ring.append(lips_outside_ring_upper)
outside_ring = np.concatenate(outside_ring, axis=0)

lips_inside_ring_lower = flame_vertices[  # black
    flame_head.mask.v.lip_inside_ring_lower].cpu().detach().numpy()
lips_inside_ring_upper = flame_vertices[
    flame_head.mask.v.lip_inside_ring_upper].cpu().detach().numpy()
inside_ring = []
if lower:
    inside_ring.append(lips_inside_ring_lower)
if upper:
    inside_ring.append(lips_inside_ring_upper)
inside_ring = np.concatenate(inside_ring, axis=0)

connection_lines = inside_ring - outside_ring

inside_mouth_ring = inside_ring + connection_lines
connection_lines = inside_mouth_ring - inside_ring

import plotly.graph_objects as go

fig = go.Figure()

if show_lips:
    fig.add_trace(
        go.Scatter3d(
            x=lips[:, 2],
            y=lips[:, 0],
            z=lips[:, 1],
            mode='markers',
            marker=dict(color='red'),
            name='Lips'))

fig.add_trace(
    go.Scatter3d(
        x=outside_ring[:, 2],
        y=outside_ring[:, 0],
        z=outside_ring[:, 1],
        mode='markers',
        marker=dict(color='green'),
        name='Outside Ring'))
fig.add_trace(
    go.Scatter3d(
        x=inside_ring[:, 2],
        y=inside_ring[:, 0],
        z=inside_ring[:, 1],
        mode='markers',
        marker=dict(color='black'),
        name='Inside Ring'))

if connect_rings:
    for i in range(connection_lines.shape[0]):
        fig.add_trace(
            go.Scatter3d(
                x=[outside_ring[i, 2], inside_ring[i, 2]],
                y=[outside_ring[i, 0], inside_ring[i, 0]],
                z=[outside_ring[i, 1], inside_ring[i, 1]],
                mode='lines',
                line=dict(color='blue'),
                name='Connection Lines'))

if inside_mouth_ring_plot:
    fig.add_trace(
        go.Scatter3d(
            x=inside_mouth_ring[:, 2],
            y=inside_mouth_ring[:, 0],
            z=inside_mouth_ring[:, 1],
            mode='markers',
            marker=dict(color='red'),
            name='Inside Mouth Ring'))

fig.update_layout(scene=dict(aspectmode='data'), title='Flame Vertices Plot')
fig.show()


In [ ]:
from thesis.code_talker.stage1_runner import Stage1Runner

ckpt = 'tb_logs/vector_quantization/default_quantization/version_0/checkpoints/epoch=199-step=15400.ckpt'
model = Stage1Runner.load_from_checkpoint(ckpt)
model = model.cuda()

In [ ]:
v = model.predict(flame_params)

In [ ]:
batch = 1
time = 100
c_in = 15429
x = torch.randn(batch, time, c_in).cuda()

In [ ]:
model.model.encoder(x).shape

In [ ]:
v = model.canonical_flame_vertices.reshape(1, -1, 3)

In [ ]:
from thesis.render_vertex_video import _generate_image

img = _generate_image(v, flame_head, time_step=0)

In [ ]:
from thesis.data_management import QuantizationDataset, UnbatchedFlameParams
from thesis.constants import TRAIN_SEQUENCES

dataset = QuantizationDataset(TRAIN_SEQUENCES, window_size=None)
idx = np.random.randint(0, len(dataset))
print(f"idx: {idx}")
flame, _, _ = dataset[idx]
flame = UnbatchedFlameParams(*flame)
flame.expr.shape

In [ ]:
from thesis.data_management import SequenceManager
from thesis.flame import FlameHead

sm = SequenceManager(3)
flame_parmas = sm.flame_params[0:1]
flame_head = FlameHead()
v = flame_head.forward(flame_parmas)
v.shape

In [ ]:
from thesis.code_talker.models.code_talker_config import QuantizerConfig, QuantizationTrainingConfig
from thesis.code_talker.stage1_runner import Stage1Runner

config = QuantizerConfig()
training_config = QuantizationTrainingConfig()
runner = Stage1Runner(config, training_config)

In [ ]:
batch_size = 2
time = 101
input_dim = config.input_dim
x = torch.randn(batch_size, time, input_dim)
template = torch.randn(batch_size, input_dim)

quant, emb_loss, info = runner.model.encode(x)
print(quant.shape)  # (batch_size, z_quant_dim, time * face_qun) # (i.e. B, C, L*F)

dec = runner.model.decode(quant)
print(dec.shape)  # (batch_size, time, input_dim)


In [ ]:
from thesis.video_utils import side_by_side

side_by_side(
    # video_gt='tmp/video_gt.mp4',
    # output_path='tmp/gt_flame_and_pred.mp4',
    video_pred='tmp/video_pred.mp4',
    video_gt='tmp/gt/sequence_100.mp4',
    output_path='tmp/gt_video_and_pred.mp4',
)

In [ ]:
from thesis.render_video import render_gt_video

render_gt_video(100)
